# Se entrena una red para detección de los bordes de etiquetas tanto de las etiquetas box como las soft

In [1]:
import tensorflow as tf
import pandas as pd
import cv2
import matplotlib.pyplot as plt
import numpy as np
import os
import albumentations as A
from functools import partial

In [2]:
tf.__version__, A.__version__

('2.3.0', '0.5.1')

In [4]:
# Si se necesita actualizar la versión de albumentation se puede ejecutar esta linea
# !pip install -U git+https://github.com/albumentations-team/albumentations

In [ ]:
path_img_resized = "/content/drive/My Drive/stamps_resized/img_resized.npy"
img_total = np.load(path_img_resized)

In [ ]:
path_etiquetas = "/content/drive/My Drive/stamps_resized/etiquetas_img_resized.npy"
et_total = np.load(path_etiquetas)

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
# Se verifica el shape de las imagenes y etiquetas
img_total.shape, et_total.shape

((1887, 224, 224, 3), (1887, 4))

In [ ]:
# se comprueba que esten normalizadas las etiquetas
et_total.max()

1.0

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(img_total, et_total, random_state=5, test_size=0.1)

In [ ]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((1698, 224, 224, 3), (189, 224, 224, 3), (1698, 4), (189, 4))

In [ ]:
from keras.applications.vgg19 import VGG19
import keras

# Se usara la red VGG 19 como primera aproximación

In [ ]:
model = VGG19(include_top=False, input_shape=(224, 224, 3))

80142336/80134624 [==============================] - 0s 0us/step


In [ ]:
for layer in model.layers:
    layer.trainable = False

In [ ]:
model.get_layer(name="block5_pool").output

<tf.Tensor 'block5_pool/MaxPool:0' shape=(None, 7, 7, 512) dtype=float32>

In [ ]:
salida1 = keras.layers.Flatten()(model.get_layer(name="block5_pool").output)
salida1 = keras.layers.Dense(255)(salida1)
salida1 = keras.layers.Activation("relu")(salida1)
salida1 = keras.layers.Dense(4, activation="sigmoid")(salida1)

In [ ]:
modelo = keras.models.Model(inputs=model.inputs, outputs=salida1)

In [ ]:
check = keras.callbacks.ModelCheckpoint("temporal_2.h5", monitor="val_mse", 
                                        verbose=1, save_best_only=True, mode='min')

In [ ]:
#adam = keras.optimizers.Adam(learning_rate=0.001)
modelo.compile(optimizer="adam", loss="binary_crossentropy", metrics=["mse"])

In [ ]:
modelo.fit(X_train, y_train, epochs=20, batch_size=12, validation_data=(X_test, y_test), callbacks=[check])

Epoch 1/20
142/142 [==============================] - ETA: 0s - loss: 0.7632 - mse: 0.0225
Epoch 00001: val_mse improved from inf to 0.01502, saving model to temporal_2.h5
142/142 [==============================] - 12s 86ms/step - loss: 0.7632 - mse: 0.0225 - val_loss: 0.6706 - val_mse: 0.0150
Epoch 2/20
142/142 [==============================] - ETA: 0s - loss: 0.6670 - mse: 0.0142
Epoch 00002: val_mse improved from 0.01502 to 0.00943, saving model to temporal_2.h5
142/142 [==============================] - 12s 86ms/step - loss: 0.6670 - mse: 0.0142 - val_loss: 0.6588 - val_mse: 0.0094
Epoch 3/20
142/142 [==============================] - ETA: 0s - loss: 0.6552 - mse: 0.0101
Epoch 00003: val_mse improved from 0.00943 to 0.00568, saving model to temporal_2.h5
142/142 [==============================] - 12s 86ms/step - loss: 0.6552 - mse: 0.0101 - val_loss: 0.6495 - val_mse: 0.0057
Epoch 4/20
142/142 [==============================] - ETA: 0s - loss: 0.6417 - mse: 0.0048
Epoch 00004: val

In [ ]:
modelo.load_weights("temporal_2.h5")

In [ ]:
modelo.evaluate(img_total, et_total)

59/59 [==============================] - 8s 143ms/step - loss: 0.6317 - mse: 2.8807e-04


[0.6316528916358948, 0.0002880691026803106]

In [ ]:
#keras.models.save_model(modelo, "modelo/deteccion_etiquetas_box_1.h5")

# La siguiente función evalua los resultados

In [ ]:
def ver_prediccion(ruta, max_bbox=182):
  i = cv2.imread(ruta)
  i = cv2.resize(i, dsize=(224, 224))
  i = np.expand_dims(i, axis=0)
  i = i/i.max()
  prediccion = modelo.predict(i)
  bbox = prediccion[0]*max_bbox
  x, y, w, h = bbox
  crop = i[:, int(y):int(y+h), int(x):int(x+w), :]
  fig, ax = plt.subplots(1, 2, figsize=(15, 15))
  ax[0].imshow(i[0])
  ax[0].set_title("Imagen suministrada")
  ax[1].imshow(crop[0])
  ax[1].set_title("Box estimado")
  plt.show()

In [ ]:
path_img = "/content/drive/My Drive/stamps_resized/gen_box_resized/"
for x in range(10):
  ver_prediccion(path_img+os.listdir(path_img)[x])

Output hidden; open in https://colab.research.google.com to view.

# Se observa buenos resultados, aunque resta hacer pruebas mas intensivas y se recomiendo que las fotos se tomen en el mismo formato mostrado

In [5]:
# Creado por Victor H. Vivas Cedeño